<a href="https://colab.research.google.com/github/lakshaykun/Recommender_System/blob/main/Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import ast

#Importing Dataset

In [ ]:
movies = pd.read_csv('/content/drive/MyDrive/Datasets/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/drive/MyDrive/Datasets/tmdb_5000_credits.csv')

In [ ]:
movies = movies[['genres', 'id', 'keywords', 'original_title', 'overview', 'popularity', 'release_date', 'revenue', 'tagline', 'title', 'vote_average']]

In [ ]:
df = movies.merge(credits)

# Data Preprocessing

In [ ]:
df.isnull().sum()

genres              0
id                  0
keywords            0
original_title      0
overview            3
popularity          0
release_date        1
revenue             0
tagline           844
title               0
vote_average        0
movie_id            0
cast                0
crew                0
dtype: int64

In [ ]:
def fetch_keys(obj):
  L = ''
  for i in ast.literal_eval(obj):
    L += i['name'] + ' '
  return L

def fetch_cast(obj):
  L = []
  counter = 0
  for i in ast.literal_eval(obj):
    if counter != 3:
      L.append(i['id'])
      counter += 1
    else:
      break
  return L

def fetch_time(obj):
  try:
    yr = ast.literal_eval(obj[:4])
    if yr > 2000:
      return 2
    else:
      return 1
  except:
    return 1.5

def fetch_dir(obj):
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      return i["id"]

In [ ]:
df['genres'] = df['genres'].apply(fetch_keys)
df['keywords'] = df['keywords'].apply(fetch_keys)
df['cast'] = df['cast'].apply(fetch_cast)
df['release_date'] = df['release_date'].apply(fetch_time)
df['crew'] = df['crew'].apply(fetch_dir)

In [ ]:
df['overview'] = df['overview'].fillna('')
df['tagline'] = df['tagline'].fillna('')
df['crew'] = df['crew'].fillna(0)

In [ ]:
df['tags'] = df['genres'] + ' ' + df['keywords'] + ' ' + df['original_title'] + ' ' + df['overview'] + ' ' + df['tagline']

In [ ]:
new_df = df[['id', 'title', 'tags', 'popularity', 'release_date', 'revenue', 'vote_average', 'cast', 'crew']]

In [ ]:
def process_text(obj):
  import re
  from nltk.stem.porter import PorterStemmer
  obj = re.sub('[^a-zA-Z]', ' ', obj)
  obj = obj.lower()
  obj = obj.split()
  ps = PorterStemmer()
  obj = [ps.stem(i) for i in obj]
  obj = ' '.join(obj)
  return obj

new_df['tags'] = new_df['tags'].apply(process_text)

<ipython-input-14-bbb097db2822>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(process_text)


In [ ]:
def extraction(obj):
  global i
  try:
    return obj[i]
  except:
    return 0

In [ ]:
for i in range(3):
  new_df[f'cast_{i}'] = new_df['cast'].apply(extraction)

<ipython-input-17-a12932b82964>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[f'cast_{i}'] = new_df['cast'].apply(extraction)
<ipython-input-17-a12932b82964>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[f'cast_{i}'] = new_df['cast'].apply(extraction)
<ipython-input-17-a12932b82964>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [ ]:
from sklearn.preprocessing import Normalizer
sc = Normalizer()
new_df[['popularity', 'release_date', 'revenue', 'vote_average', 'crew', 'cast_0', 'cast_1', 'cast_2']] = sc.fit_transform(new_df[['popularity', 'release_date', 'revenue', 'vote_average', 'crew', 'cast_0', 'cast_1', 'cast_2']])

<ipython-input-19-49c0f37aaad4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[['popularity', 'release_date', 'revenue', 'vote_average', 'crew', 'cast_0', 'cast_1', 'cast_2']] = sc.fit_transform(new_df[['popularity', 'release_date', 'revenue', 'vote_average', 'crew', 'cast_0', 'cast_1', 'cast_2']])


In [ ]:
df.head(1)

,genres,id,keywords,original_title,overview,popularity,release_date,revenue,tagline,title,vote_average,movie_id,cast,crew,tags
0,Action Adventure Fantasy Science Fiction,19995,culture clash future space war space colony so...,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,2.0,2787965087,Enter the World of Pandora.,Avatar,7.2,19995,"[65731, 8691, 10205]",2710.0,Action Adventure Fantasy Science Fiction cult...


# Bag of Words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 15000, stop_words = "english")

In [ ]:
vector = cv.fit_transform(new_df['tags']).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
temp = new_df.drop(columns = ['id', 'title', 'tags', 'cast']).values

array([[5.39596345e-08, 7.17369098e-10, 1.00000000e+00, ...,
        2.35766941e-05, 3.11732741e-06, 3.66037582e-06],
       [1.44726967e-07, 2.08116545e-09, 1.00000000e+00, ...,
        8.84495317e-08, 1.18626431e-07, 1.20707596e-07],
       [1.21925608e-07, 2.27098631e-09, 9.99999990e-01, ...,
        9.97417187e-06, 3.10205375e-05, 1.37995348e-04],
       ...,
       [1.18291799e-06, 1.63785067e-06, 0.00000000e+00, ...,
        9.71818693e-03, 2.85927780e-02, 4.78195070e-02],
       [2.79249188e-06, 6.51683969e-06, 0.00000000e+00, ...,
        2.67493460e-01, 6.84431089e-01, 6.68953595e-03],
       [1.38219508e-05, 1.43241335e-05, 0.00000000e+00, ...,
        4.98458358e-01, 6.12807915e-01, 2.17297105e-02]])

In [ ]:
final_vector = np.concatenate((vector, temp), axis = 1)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(final_vector)

In [ ]:
similarity

array([[1.00000000e+00, 1.15242196e-01, 6.18170833e-02, ...,
        7.50058784e-02, 1.35445903e-02, 2.03231317e-07],
       [1.15242196e-01, 1.00000000e+00, 7.03488720e-02, ...,
        7.11316253e-02, 3.08276782e-02, 1.61311501e-02],
       [6.18170833e-02, 7.03488720e-02, 1.00000000e+00, ...,
        3.05246862e-02, 4.10969352e-07, 4.67379226e-07],
       ...,
       [7.50058784e-02, 7.11316253e-02, 3.05246862e-02, ...,
        1.00000000e+00, 4.94866738e-02, 5.08884666e-02],
       [1.35445903e-02, 3.08276782e-02, 4.10969352e-07, ...,
        4.94866738e-02, 1.00000000e+00, 9.05255580e-02],
       [2.03231317e-07, 1.61311501e-02, 4.67379226e-07, ...,
        5.08884666e-02, 9.05255580e-02, 1.00000000e+00]])

In [ ]:
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:20]:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend("Ramanujan")

A Beautiful Mind
Good Will Hunting
The R.M.
I Origins
Three Kingdoms: Resurrection of the Dragon
Mr. Peabody & Sherman
I Come with the Rain
1982
School for Scoundrels
City By The Sea
Love Happens
Le Havre
Forsaken
Interview with the Assassin
Shine
Mr. Turner
Dil Jo Bhi Kahey...
Son of God
Blood and Wine


In [ ]:
import pickle

In [ ]:
pickle.dump(df, open('movie_list.pkl','wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))